In [8]:
import sys
sys.path.append('./models')
from models.BBSNet_model import BBSNetTransformerAttention
from models.BBSNet_model import BBSNetChannelSpatialAttention
from BBSNet_train import setup_gpu, get_model, get_data, main
from BBSNet_test import test
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# from models.RFNet import RFNet
# import torch

# rgb = torch.rand((2, 3, 352, 352)).cuda()
# depth = torch.rand((2, 1, 352, 352)).cuda()
# model = RFNet(32, 50).cuda()
# l = model(rgb, depth)

# Train

In [2]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0


/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


load data...
Start train...


/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/nn/_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


2025-09-01 12:06:11.967032 Epoch [001/002], Step [0001/2184], Loss1: 1.6067 Loss2: 1.4613
Keyboard Interrupt: save model and exit.
save checkpoints successfully!


KeyboardInterrupt: 

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [ ]:
from models.BBSNet_model import BBSNetTransformerAttention

model = BBSNetTransformerAttention()

# assume your model is called `model`
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {total_params - trainable_params}")


Total parameters: 60894418
Trainable parameters: 60894418
Non-trainable parameters: 0


In [ ]:
test(method='swin_s2') 

USE GPU 0
save img to:  ./pred/swin_s1/NJU2K/000008_left.png
save img to:  ./pred/swin_s1/NJU2K/000013_left.png
save img to:  ./pred/swin_s1/NJU2K/000017_left.png
save img to:  ./pred/swin_s1/NJU2K/000023_left.png
save img to:  ./pred/swin_s1/NJU2K/000025_left.png
save img to:  ./pred/swin_s1/NJU2K/000026_left.png
save img to:  ./pred/swin_s1/NJU2K/000027_left.png
save img to:  ./pred/swin_s1/NJU2K/000028_left.png
save img to:  ./pred/swin_s1/NJU2K/000034_left.png
save img to:  ./pred/swin_s1/NJU2K/000040_left.png
save img to:  ./pred/swin_s1/NJU2K/000043_left.png
save img to:  ./pred/swin_s1/NJU2K/000084_left.png
save img to:  ./pred/swin_s1/NJU2K/000087_left.png
save img to:  ./pred/swin_s1/NJU2K/000089_left.png
save img to:  ./pred/swin_s1/NJU2K/000090_left.png
save img to:  ./pred/swin_s1/NJU2K/000097_left.png
save img to:  ./pred/swin_s1/NJU2K/000100_left.png
save img to:  ./pred/swin_s1/NJU2K/000110_left.png
save img to:  ./pred/swin_s1/NJU2K/000111_left.png
save img to:  ./pred/

In [7]:
evaluate()

Eval [test_in_train] Dataset [Fmeasure] with [bbsnet] Method.


100%|██████████| 800/800 [00:22<00:00, 35.94it/s]




Eval [test_in_train] Dataset [MAE] with [bbsnet] Method.


100%|██████████| 800/800 [00:02<00:00, 312.44it/s]




Eval [test_in_train] Dataset [Smeasure] with [bbsnet] Method.


...



Eval [test_in_train] Dataset [Emeasure] with [swin_best2] Method.


100%|██████████| 800/800 [01:12<00:00, 11.08it/s]




Eval [test_in_train] Dataset [Fbw_measure] with [swin_best2] Method.


100%|██████████| 800/800 [00:21<00:00, 36.73it/s]



[cost:124.1648s][test_in_train Dataset] [swin_best2 Method] 0.0327 mae, 0.9122 max-fmeasure, 0.9013 mean-fmeasure, 0.9001 adp-fmeasure, 0.9495 max-Emeasure, 0.9441 mean-Emeasure, 0.9361 adp-Emeasure, 0.9142 S-measure_alpha05, 0.8774 Fbw-measure



In [ ]:
file_path = "PySODEvalToolkit-master/output/results.xlsx"

In [15]:
import openpyxl
from openpyxl.styles import PatternFill

def highlight_max_in_metrics(file_path, output_path):
    # Load the workbook and sheet
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    
    # Define the fill style for maximum values
    yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
    
    # Get the maximum row and column indices
    max_row = sheet.max_row
    max_col = sheet.max_column
    
    # Data starts from row 3 (index 3) since rows 1 and 2 are headers
    start_data_row = 3
    
    # Iterate through each column
    for col in range(1, max_col + 1):
        values = []
        # Collect all numeric values in the column
        for row in range(start_data_row, max_row + 1):
            cell = sheet.cell(row=row, column=col)
            try:
                num_val = float(cell.value)
                values.append((num_val, cell))
            except (ValueError, TypeError):
                continue
        
        if not values:
            continue
        
        # Find the maximum value in the column
        max_val = max(values, key=lambda x: x[0])[0]
        
        # Highlight all cells with the maximum value
        for value, cell in values:
            if value == max_val:
                cell.fill = yellow_fill
    
    # Save the modified workbook
    workbook.save(output_path)

# Example usage
input_file = "PySODEvalToolkit-master/output/results.xlsx"
output_file = "PySODEvalToolkit-master/output/results1.xlsx"
highlight_max_in_metrics(input_file, output_file)